In [4]:
import polars as pl
from sklearn.naive_bayes import MultinomialNB
import sys 
import os 

# Manually add path to read from another folder 
path2add = os.path.normpath(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), os.path.pardir, "utils"))
)
if not (path2add in sys.path):
    sys.path.append(path2add)

from modelling import *

In [3]:
# importing data 
df = pl.read_csv("../data/cleaned_google_analytics.csv",ignore_errors=True)
df.head()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,i64,str,str,str,str,str,str,str,str,str,str
"""2017-07-31""",1501520683,2219384770970157334,"""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
"""2017-07-31""",1501520683,2219384770970157334,"""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
"""2017-07-31""",1501520683,2219384770970157334,"""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Sport Bag""","""Home/Bags/More Bags/"""
"""2017-07-31""",1501520683,2219384770970157334,"""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
"""2017-07-31""",1501520683,2219384770970157334,"""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""


In [29]:
event_df.filter(pl.col('visitorid') == 1308708)

timestamp,visitorid,event,itemid,transactionid
i64,i64,str,i64,str
1434389834443,1308708,"""transaction""",181973,"""3529"""
1434391470175,1308708,"""view""",260955,null
1434389604062,1308708,"""view""",181973,null
1434389512630,1308708,"""view""",169256,null
1434389748591,1308708,"""addtocart""",181973,null


In [30]:
# Create list of customers who made a purchase
customers = event_df.filter(pl.col("transactionid").is_not_null()).select("visitorid").unique().to_series().to_list()
customers[:10]

[1076167,
 510429,
 1276854,
 109850,
 372661,
 597988,
 1203461,
 13764,
 1311727,
 1128088]

In [31]:
# Creating a list of lists for every item a customer purchased 
purchases = []

for i in customers:
        items = event_df.filter((pl.col("visitorid") == pl.lit(i)) & (pl.col("transactionid").is_not_null())
                                ).select("itemid").to_series().to_list()    
        purchases.append(items)

In [32]:
purchases

[[172103],
 [461686],
 [293808, 58846],
 [329408],
 [339517],
 [364750],
 [132683],
 [8096],
 [7341, 217642],
 [44402],
 [307588],
 [216842],
 [52039],
 [342367],
 [223203],
 [396732,
  176669,
  456858,
  334401,
  1684,
  456858,
  176669,
  253214,
  358927,
  176669,
  334401,
  253214,
  1684],
 [418609],
 [226628],
 [417927],
 [416170],
 [294438, 142819],
 [255725, 255725, 255725],
 [250891],
 [256473],
 [90293],
 [262813],
 [83837],
 [294438],
 [272596],
 [379660],
 [363368],
 [163297],
 [11976],
 [371794],
 [51650],
 [179601],
 [19976],
 [461686],
 [134484],
 [348938],
 [280065],
 [73852, 73852, 320185, 320185, 355391, 355391],
 [165169],
 [334094],
 [402295],
 [227790],
 [51629],
 [445776],
 [144540],
 [262982],
 [443047, 257059],
 [57445],
 [369447],
 [126890],
 [51354, 280375, 210087, 73602],
 [325611],
 [247420, 277139],
 [244838],
 [409655],
 [315158],
 [276494],
 [358895],
 [340375],
 [346088],
 [382419],
 [13925, 13925],
 [139522],
 [380971,
  85771,
  172894,
  449220,


In [33]:
# Recommend 5 items that were bought by previous customers 
# Who viewed item 31640
new_customer_recommender(31640, purchases)

[424193, 246532, 217605, 149767, 124938]

In [5]:
df.head()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,i64,str,str,str,str,str,str,str,str,str,str
"""2017-07-31""",1501520683,2219384770970157334,"""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
"""2017-07-31""",1501520683,2219384770970157334,"""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
"""2017-07-31""",1501520683,2219384770970157334,"""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Sport Bag""","""Home/Bags/More Bags/"""
"""2017-07-31""",1501520683,2219384770970157334,"""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
"""2017-07-31""",1501520683,2219384770970157334,"""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""


In [53]:
df = df.with_columns(pl.when(pl.col('transactionId') != 'null').then(1).otherwise(0).alias("transactionId"))

In [54]:
df

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory,transactionId_indicator
str,i64,i64,str,str,str,str,str,str,str,i32,str,str,i32
"""2017-07-31""",1501520683,2219384770970157334,"""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Google Lunch Bag""","""Home/Bags/More Bags/""",0
"""2017-07-31""",1501520683,2219384770970157334,"""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/""",0
"""2017-07-31""",1501520683,2219384770970157334,"""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Sport Bag""","""Home/Bags/More Bags/""",0
"""2017-07-31""",1501520683,2219384770970157334,"""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/""",0
"""2017-07-31""",1501520683,2219384770970157334,"""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/""",0
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2017-07-31""",1501537217,7483600664917507409,"""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""",1,"""Android 17oz Stainless Steel S…","""Drinkware""",1
"""2017-07-31""",1501537217,7483600664917507409,"""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""",1,"""Android Men's Short Sleeve Her…","""Apparel""",1
"""2017-07-31""",1501537217,7483600664917507409,"""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""",1,"""Android Men's Take Charge Shor…","""Apparel""",1


In [18]:
df.select(pl.col('country'),pl.col('browser'),pl.col('operatingSystem'),pl.col('source'),pl.col('v2ProductCategory'))

country,browser,operatingSystem,source,v2ProductCategory
str,str,str,str,str
"""Slovakia""","""Firefox""","""Windows""","""google""","""Home/Bags/More Bags/"""
"""Slovakia""","""Firefox""","""Windows""","""google""","""Home/Bags/More Bags/"""
"""Slovakia""","""Firefox""","""Windows""","""google""","""Home/Bags/More Bags/"""
"""Slovakia""","""Firefox""","""Windows""","""google""","""Home/Bags/More Bags/"""
"""Slovakia""","""Firefox""","""Windows""","""google""","""Home/Bags/More Bags/"""
…,…,…,…,…
"""United States""","""Chrome""","""Macintosh""","""(direct)""","""Drinkware"""
"""United States""","""Chrome""","""Macintosh""","""(direct)""","""Apparel"""
"""United States""","""Chrome""","""Macintosh""","""(direct)""","""Apparel"""


In [30]:
X = df['transactionId_indicator']
y = df.select(pl.col('country'),pl.col('browser'),pl.col('operatingSystem'),pl.col('source'),pl.col('v2ProductCategory'))
X

transactionId_indicator
i32
0
0
0
0
0
…
1
1
1


In [35]:
X = X.to_numpy()

In [36]:
X = X.reshape(-1,1)
X

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [0]])